In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

In [2]:
from utils import solr
from utils.logger import logger as log

log.info('START')

ROWS = solr.select('experiment', {'params':{'q':'*:*', 'rows':0}})['response']['numFound']

print(ROWS)

2023-02-05 13:56:00,805 - INFO - START
82421326


In [3]:
data = solr.select('experiment', 
{
    'params':
    {
        'q':'*:*',
        'wt':'csv',
        'rows': 0,
        'facet':''
    }
})

fl = pd.Series(data.decode('utf-8').split(','))

fl = ','.join(fl.sort_values().str.strip().tolist())

fl.split(',')

['_version_',
 'age_in_days',
 'age_in_weeks',
 'allele_accession_id',
 'allele_symbol',
 'allelic_composition',
 'biological_sample_group',
 'category',
 'colony_id',
 'data_point',
 'datasource_name',
 'date_of_birth',
 'date_of_experiment',
 'developmental_stage_acc',
 'developmental_stage_name',
 'discrete_point',
 'download_file_path',
 'experiment_id',
 'experiment_source_id',
 'external_sample_id',
 'file_type',
 'gene_accession_id',
 'gene_symbol',
 'genetic_background',
 'id',
 'image_link',
 'increment_value',
 'life_stage_acc',
 'life_stage_name',
 'litter_id',
 'metadata',
 'metadata_group',
 'observation_id',
 'observation_type',
 'parameter_association_name',
 'parameter_association_sequence_id',
 'parameter_association_stable_id',
 'parameter_association_value',
 'parameter_name',
 'parameter_stable_id',
 'phenotyping_center',
 'pipeline_name',
 'pipeline_stable_id',
 'procedure_group',
 'procedure_name',
 'procedure_sequence_id',
 'procedure_stable_id',
 'production_cen

In [4]:
q = 'gene_symbol:[* TO *] OR gene_accession_id:[* TO *] OR allele_symbol:[* TO *] OR allele_accession_id:[* TO *]'
     
fl = 'gene_symbol, gene_accession_id, allele_symbol, allele_accession_id, zygosity'

data = solr.select('experiment', 
{
    'params':
    {
        'q':q,
        'rows': ROWS,
        'fl':fl
    }
})

log.info('Create dataframe.')

df = pd.DataFrame(data['response']['docs'])

df = df.drop_duplicates()

log.info('Write to file.')

df.to_pickle('./data/gene_allele_all.pkl')

2023-02-05 14:40:33,417 - INFO - Create dataframe.
2023-02-05 14:42:43,979 - INFO - Write to file.


In [15]:
df = pd.read_pickle('./data/gene_allele_all.pkl')

df = df.loc[~df['allele_accession_id'].str.contains(r'NOT-RELEASED')]

df = df.drop(['zygosity'], axis=1)

df = df.drop_duplicates()

df.loc[df['gene_symbol'].duplicated()]

,allele_accession_id,gene_accession_id,gene_symbol,allele_symbol
213,MGI:5548324,MGI:101876,Tead1,Tead1<tm1b(KOMP)Wtsi>
365,MGI:4434811,MGI:1916658,Dnajc17,Dnajc17<tm1a(EUCOMM)Hmgu>
377,MGI:3621117,MGI:108078,Sfrp2,Sfrp2<tm1Aksh>
420,MGI:5692920,MGI:98894,Ucp1,Ucp1<tm1b(EUCOMM)Hmgu>
422,MGI:5501061,MGI:1931838,Dbn1,Dbn1<tm1b(KOMP)Wtsi>
629,MGI:1862019,MGI:1100883,Celsr1,Celsr1<Scy>
697,MGI:3797584,MGI:1924337,Ankrd11,Ankrd11<Yod>
738,MGI:3838355,MGI:1336167,Prkab1,Prkab1<Gt(RRR454)Byg>
790,MGI:4431702,MGI:1919305,Slc38a10,Slc38a10<tm1a(EUCOMM)Wtsi>
870,MGI:5548492,MGI:1859162,Rnf10,Rnf10<tm1b(KOMP)Wtsi>
